# 구글 드라이브 연동하기
모델 체크포인트 등을 저장해 둘 구글 드라이브를 연결합니다. 자신의 구글 계정에 적용됩니다.

In [ ]:
import os
import pandas as pd
import numpy as np
import re
import seaborn as sns

In [ ]:
!pip install fasttext

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 68 kB 2.3 MB/s 
  Using cached pybind11-2.10.1-py3-none-any.whl (216 kB)
  Created wheel for fasttext: filename=fasttext-0.9.2-cp38-cp38-linux_x86_64.whl size=3132503 sha256=bb8c23675a51fe316579def09cdb5cbbcaad54a63acc20fd71604841f4544116
  Stored in directory: /root/.cache/pip/wheels/93/61/2a/c54711a91c418ba06ba195b1d78ff24fcaad8592f2a694ac94
Successfully built fasttext


In [ ]:
from google.colab import drive
drive.mount('/gdrive', force_remount=True)

Mounted at /gdrive


# 자모 단위 FASTTEXT

In [ ]:
!pip install hgtk

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Created wheel for hgtk: filename=hgtk-0.1.3-py2.py3-none-any.whl size=6688 sha256=40e5cc5178cc16b4f3d0626c5da017d81b0eaca43de14b50cff0855938797004
  Stored in directory: /root/.cache/pip/wheels/93/33/b8/bc2256172a415340e34f3c11ef2b0f3f391769000bb74de988
Successfully built hgtk


In [ ]:
!pip install konlpy

from konlpy.tag import Mecab
!git clone https://github.com/SOMJANG/Mecab-ko-for-Google-Colab.git
%cd Mecab-ko-for-Google-Colab/
!bash install_mecab-ko_on_colab190912.sh

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 19.4 MB 5.8 MB/s 
     |████████████████████████████████| 465 kB 39.0 MB/s 
Cloning into 'Mecab-ko-for-Google-Colab'...
remote: Enumerating objects: 115, done.
remote: Counting objects: 100% (24/24), done.
remote: Compressing objects: 100% (20/20), done.
remote: Total 115 (delta 11), reused 10 (delta 3), pack-reused 91
Receiving objects: 100% (115/115), 1.27 MiB | 9.71 MiB/s, done.
Resolving deltas: 100% (50/50), done.
/content/Mecab-ko-for-Google-Colab
Installing konlpy.....
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Done
Installing mecab-0.996-ko-0.9.2.tar.gz.....
from https://bitbucket.org/eunjeon/mecab-ko/downloads/mecab-0.996-ko-0.9.2.tar.gz
--2023-01-01 16:52:47--  https://bitbucket.org/eunjeon/mecab-ko/downloads/mecab-0.996-ko-0.9.2.tar.gz
Resolving bitbucket.org (bitbucket.org)... 104.1

In [ ]:
import hgtk
from tqdm import tqdm
from konlpy.tag import Mecab

def word_to_jamo(token):
    def to_special_token(jamo):
      if not jamo:
        return '-'
      else:
        return jamo
    decomposed_token = ''

    for char in token:
        try:
        # char( 음 절 ) 을 초 성 , 중 성 , 종 성 으 로 분 리
            cho, jung, jong = hgtk.letter.decompose(char)

            # 자 모 가 빈 문 자 일 경 우 특 수 문 자 - 로 대 체
            cho = to_special_token(cho)
            jung = to_special_token(jung)
            jong = to_special_token(jong)
            decomposed_token = decomposed_token + cho + jung + jong

        # 만 약 char( 음 절 ) 이 한 글 이 아 닐 경 우 자 모 를 나 누 지 않 고 추 가
        except Exception as exception:
            if type(exception).__name__ == 'NotHangulException':
                decomposed_token += char

    # 단 어 토 큰 의 자 모 단 위 분 리 결 과 를 추 가
    return decomposed_token


mecab = Mecab()

def tokenize_by_jamo(s):
    return [word_to_jamo(token) for token in mecab.morphs(s)]

In [ ]:
from sklearn.model_selection import train_test_split

df = pd.read_csv("/gdrive/MyDrive/2023beaver/final.csv")

train_data, test_data = train_test_split(df, test_size=0.20, random_state=2)
train_data, val_data = train_test_split(train_data, test_size=0.25, random_state=2)

In [ ]:
df['title'][0]

'국내산 100% 전라도 배추 김치 포기 김장김치 주문'

In [ ]:
word_to_jamo(df['title'][0])

'ㄱㅜㄱㄴㅐ-ㅅㅏㄴ 100% ㅈㅓㄴㄹㅏ-ㄷㅗ- ㅂㅐ-ㅊㅜ- ㄱㅣㅁㅊㅣ- ㅍㅗ-ㄱㅣ- ㄱㅣㅁㅈㅏㅇㄱㅣㅁㅊㅣ- ㅈㅜ-ㅁㅜㄴ'

In [ ]:
print(tokenize_by_jamo(df['title'][0]))

['ㄱㅜㄱㄴㅐ-ㅅㅏㄴ', '100', '%', 'ㅈㅓㄴㄹㅏ-ㄷㅗ-', 'ㅂㅐ-ㅊㅜ-', 'ㄱㅣㅁㅊㅣ-', 'ㅍㅗ-ㄱㅣ-', 'ㄱㅣㅁㅈㅏㅇ', 'ㄱㅣㅁㅊㅣ-', 'ㅈㅜ-ㅁㅜㄴ']


In [ ]:
def tokenized(data):
  tokenized_data=[]

  for sample in tqdm(data['title'].to_list()):
      tokenzied_sample = tokenize_by_jamo(sample) # 자 소 단 위 토 큰 화
      tokenized_data.append(tokenzied_sample)

  return tokenized_data

In [ ]:
def jamo_to_word(jamo_sequence):
    tokenized_jamo = []
    index = 0

# 1. 초 기 입 력
# jamo_sequence = ' ﾤ ￂ ﾱ ﾧ ￌ ﾷ ﾵ ￃ ﾷ '

    while index < len(jamo_sequence):
    # 문 자 가 한 글 ( 정 상 적 인 자 모 ) 이 아 닐 경 우
        if not hgtk.checker.is_hangul(jamo_sequence[index]):
            tokenized_jamo.append(jamo_sequence[index])
            index = index + 1

    # 문 자 가 정 상 적 인 자 모 라 면 초 성 , 중 성 , 종 성 을 하 나 의 토 큰 으 로 간 주 .
        else:
            tokenized_jamo.append(jamo_sequence[index:index + 3])
            index = index + 3

# 2. 자 모 단 위 토 큰 화 완 료
# tokenized_jamo : [' ﾤ ￂ ﾱ ', ' ﾧ ￌ ﾷ ', ' ﾵ ￃ ﾷ ']

        word = ''
        try:
            for jamo in tokenized_jamo:
            # 초 성 , 중 성 , 종 성 의 묶 음 으 로 추 정 되 는 경 우
                if len(jamo) == 3:
                    if jamo[2] == "-":
                    # 종 성 이 존 재 하 지 않 는 경 우
                        word = word + hgtk.letter.compose(jamo[0], jamo[1])
                    else:
                # 종 성 이 존 재 하 는 경 우
                        word = word + hgtk.letter.compose(jamo[0], jamo[1], jamo[2])
                # 한 글 이 아 닌 경 우
                else:
                    word = word + jamo

            # 복 원 중 (hgtk.letter.compose) 에 러 발 생 시 초 기 입 력 리 턴 .
            # 복 원 이 불 가 능 한 경 우 예 시 ) ' ﾤ ! ﾱ ﾧ ￌ ﾷ ﾵ ￃ ﾷ '
        except Exception as exception:
            if type(exception).__name__ == 'NotHangulException':
                return jamo_sequence

        # 3. 단 어 로 복 원 완 료
        # word : ' 남 동 생 '

    return word

# 학습데이터 구축
학습데이터를 만듭니다.

In [ ]:
train_data.reset_index(inplace=True)
val_data.reset_index(inplace=True)
test_data.reset_index(inplace=True)

In [ ]:
train_token = tokenized(train_data)
val_token = tokenized(val_data)
test_token = tokenized(test_data)
df_token = tokenized(df)

100%|██████████| 265035/265035 [00:48<00:00, 5521.51it/s]


In [ ]:
with open('train_data.txt', 'w') as out:
    for i in tqdm(range(len(train_token)), unit=' line'):
          out.write("__label__" + train_data['category'][i] + "\t" + ' '.join(train_token[i]) + '\n')

with open('val_data.txt', 'w') as out:
    for i in tqdm(range(len(val_token)), unit=' line'):
        out.write("__label__" + val_data['category'][i] + "\t" + ' '.join(val_token[i]) + '\n')

with open('test_data.txt', 'w') as out:
    for i in tqdm(range(len(test_token)), unit=' line'):
        out.write("__label__" + test_data['category'][i] + "\t" + ' '.join(test_token[i]) + '\n')

100%|██████████| 53007/53007 [00:00<00:00, 117206.43 line/s]


In [ ]:
with open('df_data.txt', 'w') as out:
    for i in tqdm(range(len(df_token)), unit=' line'):
        out.write("__label__" + df['category'][i] + "\t" + ' '.join(df_token[i]) + '\n')

100%|██████████| 265035/265035 [00:02<00:00, 122179.43 line/s]


# Gensim을 통한 학습

In [ ]:
!pip install gensim --upgrade

In [ ]:
!pip install numpy --upgrade

In [ ]:
from gensim.models import FastText
from gensim.utils import tokenize
from gensim import utils
from gensim.test.utils import datapath

class MyIter:
    def __iter__(self):
        path = datapath('/content/test_data.txt')
        with utils.open(path, 'r', encoding='utf-8') as fin:
            for line in fin:
                yield list(tokenize(line))

model3 = FastText(vector_size=4, window=3, min_count=1)
model3.build_vocab(corpus_iterable=MyIter())
total_examples = model3.corpus_count
model3.train(corpus_iterable=MyIter(), total_examples=total_examples, epochs=5)

(3276530, 4152075)

### 저장

In [ ]:
model_fname = 'fasttext'

In [ ]:
model3.save(model_fname)
# https://projector.tensorflow.org/ 에서 시각화 하기 위해 모델을 따로 저장
model3.wv.save_word2vec_format(model_fname + "_vis")

In [ ]:
model3 = FastText.load(model_fname)

In [ ]:
def transform_gensim(word_sequence):
    return [(jamo_to_word(word), similarity) for (word, similarity) in word_sequence]

In [ ]:
transform_gensim(model3.wv.most_similar(word_to_jamo("찜닭"), topn=5))

[('맛있', 0.9995765686035156),
 ('냉동', 0.9995282292366028),
 ('냉장육', 0.9994854927062988),
 ('목살', 0.9984344840049744),
 ('통살', 0.9979778528213501)]

In [ ]:
transform_gensim(model3.wv.most_similar(word_to_jamo("볶음밥"), topn=5))

[('덮밥', 0.9965663552284241),
 ('즉', 0.996501088142395),
 ('닭볶음밥', 0.996482253074646),
 ('쌈밥', 0.9961134791374207),
 ('맙', 0.9931130409240723)]

업데이트

In [ ]:
path = datapath('/content/train_data.txt')
old_vector = []
with utils.open(path, 'r', encoding='utf-8') as fin:
    for line in fin:
      old_vector.append(line)

In [ ]:
path = datapath('/content/val_data.txt')
new_vector = []
with utils.open(path, 'r', encoding='utf-8') as fin:
    for line in fin:
      new_vector.append(line)

In [ ]:
import numpy as np
np.allclose(old_vector, new_vector, atol=1e-4)

In [ ]:
model3.build_vocab(new_vector, update=True)
model3.train(new_vector, total_examples=len(new_vector), epochs=model3.epochs)

(6759131, 21611110)

# 학습


In [ ]:
import fasttext

model = fasttext.train_supervised(input='train_data.txt', autotuneValidationFile='val_data.txt')

In [ ]:
model.predict(word_to_jamo("강아지"))

(('__label__축산물',), array([0.99634981]))

찜닭을 "과자/떡/베이커리" 로 구분하는 대단한 수준 

# 테스트
결과는 (샘플, 정확도, 재현율)

In [ ]:
model.test("test_data.txt")

(53007, 0.8575093855528515, 0.8575093855528515)

In [ ]:
model.save_model("fasttext_supervised_basemodel.bin") # 모 델 저 장

In [ ]:
import fasttext
model2 = fasttext.train_unsupervised('df_data.txt', model='skipgram')

In [ ]:
model2.get_nearest_neighbors(word_to_jamo("찜닭"), k=10)

[(0.8166288733482361, 'ㅊㅗㄴㄷㅏㄺ'),
 (0.791918158531189, 'ㅅㅐㅇㄷㅏㄺ'),
 (0.744603157043457, 'ㄷㅏㄺㅉㅣㅁ'),
 (0.7373501062393188, 'ㅌㅗ-ㅈㅗㅇㄷㅏㄺ'),
 (0.7371629476547241, 'ㄷㅏㄺ'),
 (0.7238197922706604, 'ㅌㅗㅇㄷㅏㄺ'),
 (0.7185007333755493, 'ㅉㅣㅁㅈㅣㄹㅂㅏㅇ'),
 (0.7107194066047668, 'ㄴㅓㄼㅈㅓㄱㄷㅏ-ㄹㅣ-'),
 (0.7090702652931213, 'ㅂㅜㄱㅊㅐ-'),
 (0.7037075161933899, 'ㅍㅏ-ㄷㅏㄺ')]

In [ ]:
def transform(word_sequence):
    return [(jamo_to_word(word), similarity) for (similarity, word) in word_sequence]

가장 가까운 단어(유사도 기준)

In [ ]:
print(transform(model2.get_nearest_neighbors(word_to_jamo("호떡"), k=10)))

[('소떡', 0.7843364477157593), ('빈대떡', 0.7055262327194214), ('설기떡', 0.6995227932929993), ('부떡', 0.695621132850647), ('개떡', 0.6917065978050232), ('가래떡', 0.6824575662612915), ('떡방아', 0.6752732992172241), ('호박떡', 0.6735275983810425), ('찰떡', 0.65810227394104), ('쫀떡쫀떡', 0.654928982257843)]


유추(의미 기준)

In [ ]:
transform(model2.get_analogies(word_to_jamo("치킨"), word_to_jamo("양념"), word_to_jamo("마늘")))

[('통마늘', 0.5720704197883606),
 ('마요라', 0.5677814483642578),
 ('마노', 0.5311671495437622),
 ('마요', 0.5297998785972595),
 ('마왕', 0.5232638716697693),
 ('마니', 0.5161885619163513),
 ('마늘빵', 0.5087692141532898),
 ('마마', 0.5082238912582397),
 ('마니커', 0.5073510408401489),
 ('마', 0.5069550275802612)]

# 저장된 모델 DB에 삽입 과정 

BLOB 방식은 안된다. bin을 BLOB로 파싱하는데 무리가 있다

In [ ]:
!pip install config

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
import time
import pandas as pd
import psycopg2
import numpy
from psycopg2.extensions import register_adapter, AsIs
  
def addapt_numpy_float64(numpy_float64):
    return AsIs(numpy_float64)
def addapt_numpy_int64(numpy_int64):
    return AsIs(numpy_int64)
register_adapter(numpy.float64, addapt_numpy_float64)
register_adapter(numpy.int64, addapt_numpy_int64)

conn = None
try:
    # connect to the PostgreSQL server
    host = 'arjuna.db.elephantsql.com'
    user = 'kwcuclpe'
    password = ''
    database = 'kwcuclpe'

    conn = psycopg2.connect(
        host=host,
        user=user,
        password=password,
        database=database
    )
  
    # Creating a cursor with name cur.
    cur = conn.cursor()
    cur.execute(
        "CREATE TABLE blob_datastore (s_no serial, file_name VARCHAR ( 50 ), blob_data bytea)")
    # SQL query to insert data into the database.
    insert_script = '''
        INSERT INTO blob_datastore(s_no,file_name,blob_data) VALUES (%s,%s,%s);
    '''
  
    # open('File,'rb').read() is used to read the file.
    # where open(File,'rb').read() will return the binary data of the file.
    # psycopg2.Binary(File_in_Bytes) is used to convert the binary data to a BLOB data type.
    BLOB_1 = psycopg2.Binary(
        open('/content/fasttext_supervised_basemodel.bin', 'rb').read())       # Video
    # BLOB_2 = psycopg2.Binary(
    #     open('files\Octa.jpg', 'rb').read())        # Image
    # BLOB_3 = psycopg2.Binary(open('files\Type.gif', 'rb').read())        # GIF
    # BLOB_4 = psycopg2.Binary(open('files\BlobNotes.pdf', 'rb').read())   # PDF
  
    # And Finally we pass the above mentioned values to the insert_script variable.
    insert_values = [(1, 'fasttext_basemodel', BLOB_1)]
  
    # The execute() method with the insert_script & insert_value as argument.
    for insert_value in insert_values:
        cur.execute(insert_script, insert_value)
        print(insert_value[0], insert_value[1],
              "[Binary Data]", "row Inserted Successfully")
  
    # SQL query to fetch data from the database.
    cur.execute('SELECT * FROM BLOB_DataStore')
  
    # open(file,'wb').write() is used to write the binary data to the file.
    for row in cur.fetchall():
        BLOB = row[2]
        open("new"+row[1], 'wb').write(BLOB)
        print(row[0], row[1], "BLOB Data is saved in Current Directory")
  
    # Close the connection
    cur.close()
  
except(Exception, psycopg2.DatabaseError) as error:
    print(error)
finally:
    if conn is not None:
        
        # Commit the changes to the database
        conn.commit()

이에 따라 워드임베딩만 fastText로 진행하고, 모델은 SVM을 사용하여 피클링을 시도해본다.

In [ ]:
# -*- coding: utf-8 -*-

## 환경설정
import warnings
warnings.filterwarnings('ignore')

import pandas as pd
import numpy as np
import nltk

In [ ]:
data_df = pd.read_csv("/gdrive/MyDrive/2023beaver/final.csv")

In [ ]:
data_df

,Unnamed: 0,title,category,sub_category,price
0,0,국내산 100% 전라도 배추 김치 포기 김장김치 주문,김치,포기김치,15900
1,1,피코크 조선호텔 포기김치 8kg,김치,포기김치,51040
2,2,대상 종가집 행복이온 포기김치 10kg,김치,포기김치,34200
3,3,대상 종가집 전라도 행복이온 포기김치 10kg,김치,포기김치,46840
4,4,선화동매운실비김치 맛있게 매운 대전 선화동 실비 김치 800g,김치,포기김치,11080
...,...,...,...,...,...
265030,266352,먹템 매콤한 낙곱새 극락 밀키트(2-3인분) 4팩 캠핑음식 용밀 간편조리식품 키트,밀키트,찌개/국,63370
265031,266353,[연말축제 1+1] 청정원 호밍스 한우진곰탕 450g X 10개,밀키트,찌개/국,25500
265032,266354,프레시지 프레시지 간편한 직화용기 꼬치어묵탕 818g,밀키트,찌개/국,11860
265033,266355,아임셰프 마라 밀푀유나베 밀키트 쿠킹박스 (2인분),밀키트,찌개/국,16900


In [ ]:
from sklearn.model_selection import train_test_split

train_corpus, test_corpus, train_label_names, test_label_names =\
                                 train_test_split(np.array(data_df['title']), np.array(data_df['sub_category']),
                                                       test_size=0.33, random_state=42)

train_corpus.shape, test_corpus.shape

((177573,), (87462,))

In [ ]:
def tokenized(data):
  tokenized_data=[]

  for sample in tqdm(data):
      tokenzied_sample = tokenize_by_jamo(sample) # 자 소 단 위 토 큰 화
      tokenized_data.append(tokenzied_sample)

  return tokenized_data

In [ ]:
tokenized_train = tokenized(train_corpus)
                   
tokenized_test = tokenized(test_corpus)

100%|██████████| 87462/87462 [00:15<00:00, 5509.71it/s]


In [ ]:
from gensim.models.fasttext import FastText

ft_num_features = 1000
# sg decides whether to use the skip-gram model (1) or CBOW (0)
ft_model = FastText(tokenized_train, size=ft_num_features, window=100, 
                    min_count=2, sample=1e-3, sg=1, iter=5, workers=10)

In [ ]:
model_fname="/gdrive/MyDrive/2023beaver/fastText_1000"
ft_model.save(model_fname)
# https://projector.tensorflow.org/ 에서 시각화 하기 위해 모델을 따로 저장
ft_model.wv.save_word2vec_format(model_fname + "_vis")

In [ ]:
def document_vectorizer(corpus, model, num_features):
    vocabulary = set(model.wv.index2word)
    
    def average_word_vectors(words, model, vocabulary, num_features):
        feature_vector = np.zeros((num_features,), dtype="float64")
        nwords = 0.
        
        for word in words:
            if word in vocabulary: 
                nwords = nwords + 1.
                feature_vector = np.add(feature_vector, model.wv[word])
        if nwords:
            feature_vector = np.divide(feature_vector, nwords)

        return feature_vector

    features = [average_word_vectors(tokenized_sentence, model, vocabulary, num_features)
                    for tokenized_sentence in corpus]
    return np.array(features)

In [ ]:
# generate averaged word vector features from word2vec model
avg_ft_train_features = document_vectorizer(corpus=tokenized_train, model=ft_model,
                                                     num_features=ft_num_features)
avg_ft_test_features = document_vectorizer(corpus=tokenized_test, model=ft_model,
                                                    num_features=ft_num_features)

In [ ]:
print('FastText model:> Train features shape:', avg_ft_train_features.shape, 
      ' Test features shape:', avg_ft_test_features.shape)

FastText model:> Train features shape: (177573, 1000)  Test features shape: (87462, 1000)


In [ ]:
from sklearn.model_selection import cross_val_score
from sklearn.linear_model import SGDClassifier

svm = SGDClassifier(loss='hinge', penalty='l2', random_state=42, max_iter=500)
svm.fit(avg_ft_train_features, train_label_names)
svm_ft_cv_scores = cross_val_score(svm, avg_ft_train_features, train_label_names, cv=5)
svm_ft_cv_mean_score = np.mean(svm_ft_cv_scores)
print('CV Accuracy (5-fold):', svm_ft_cv_scores)
print('Mean CV Accuracy:', svm_ft_cv_mean_score)
svm_ft_test_score = svm.score(avg_ft_test_features, test_label_names)
print('Test Accuracy:', svm_ft_test_score)

CV Accuracy (5-fold): [0.79904266 0.79321413 0.79709982 0.79616489 0.79855832]
Mean CV Accuracy: 0.7968159635264409
Test Accuracy: 0.7967917495598088


In [ ]:
predict_feature = document_vectorizer(corpus=word_to_jamo("호떡"), model=ft_model,
                                                     num_features=ft_num_features)

In [ ]:
predict_feature

array([[ 0.        ,  0.        ,  0.        ,  0.        ],
       [ 0.        ,  0.        ,  0.        ,  0.        ],
       [-0.23585831, -0.09001003, -0.63167715,  0.06523085],
       [ 0.        ,  0.        ,  0.        ,  0.        ],
       [ 0.        ,  0.        ,  0.        ,  0.        ],
       [ 0.        ,  0.        ,  0.        ,  0.        ]])

In [ ]:
svm.predict(predict_feature)

array(['차류', '차류', '차류', '차류', '차류', '차류'], dtype='<U11')